# Introduzione a Hugging Face

Hugging Face è una comunità e una piattaforma per l'apprendimento automatico. Offre una vasta gamma di modelli pre-addestrati, set di dati e strumenti per semplificare lo sviluppo e la condivisione di modelli di intelligenza artificiale.

In particolare, Hugging Face è utilizzzata per hostare molti modelli di elaborazione del linguaggio naturale (NLP), come BERT, GPT-2 e molti altri. Questi modelli possono essere utilizzati per una varietà di attività, tra cui la classificazione del testo, la traduzione automatica e la generazione di testo.

La piattaforma Hugging Face è progettata per essere facile da usare e accessibile sia ai principianti che agli esperti.

Offre un'interfaccia web intuitiva e un'API potente per interagire con i modelli e i set di dati.

In questo notebook utilizzeremo BART (large-sized model), un modello pre-addestrato sulla lingua inglese e perfezionato (fine-tuned) sul dataset CNN Daily Mail, noto per le coppie di testo e riassunto.

BART, presentato nell’articolo BART: Denoising Sequence-to-Sequence Pre-training for Natural Language Generation, Translation, and Comprehension di Lewis et al., è un modello seq2seq basato su Transformer, con un encoder bidirezionale (simile a BERT) e un decoder autoregressivo (simile a GPT). Viene addestrato corrompendo il testo e imparando a ricostruirlo.

È particolarmente efficace per la generazione di testo (es. riassunti, traduzioni), ma funziona bene anche per classificazione e question answering. Il modello che useremo è stato ottimizzato specificamente per i riassunti del CNN Daily Mail.

In [ ]:
!pip install -qU huggingface_hub transformers accelerate sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 63.0 MB/s eta 0:00:00


Questo codice permette di autenticarsi con Hugging Face per accedere ai modelli e ai set di dati.

In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from transformers import pipeline

# Qui carichiamo un modello si summarization.
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [12]:
ARTICLE = """ New York (CNN)When Liana Barrientos was 23 years old, she got married in Westchester County, New York.
A year later, she got married again in Westchester County, but to a different man and without divorcing her first husband.
Only 18 days after that marriage, she got hitched yet again. Then, Barrientos declared "I do" five more times, sometimes only within two weeks of each other.
In 2010, she married once more, this time in the Bronx. In an application for a marriage license, she stated it was her "first and only" marriage.
Barrientos, now 39, is facing two criminal counts of "offering a false instrument for filing in the first degree," referring to her false statements on the
2010 marriage license application, according to court documents.
Prosecutors said the marriages were part of an immigration scam.
On Friday, she pleaded not guilty at State Supreme Court in the Bronx, according to her attorney, Christopher Wright, who declined to comment further.
After leaving court, Barrientos was arrested and charged with theft of service and criminal trespass for allegedly sneaking into the New York subway through an emergency exit, said Detective
Annette Markowski, a police spokeswoman. In total, Barrientos has been married 10 times, with nine of her marriages occurring between 1999 and 2002.
All occurred either in Westchester County, Long Island, New Jersey or the Bronx. She is believed to still be married to four men, and at one time, she was married to eight men at once, prosecutors say.
Prosecutors said the immigration scam involved some of her husbands, who filed for permanent residence status shortly after the marriages.
Any divorces happened only after such filings were approved. It was unclear whether any of the men will be prosecuted.
The case was referred to the Bronx District Attorney\'s Office by Immigration and Customs Enforcement and the Department of Homeland Security\'s
Investigation Division. Seven of the men are from so-called "red-flagged" countries, including Egypt, Turkey, Georgia, Pakistan and Mali.
Her eighth husband, Rashid Rajput, was deported in 2006 to his native Pakistan after an investigation by the Joint Terrorism Task Force.
If convicted, Barrientos faces up to four years in prison.  Her next court appearance is scheduled for May 18.
"""

print(summarizer(ARTICLE, max_length=130, min_length=30, do_sample=False))

[{'summary_text': 'Liana Barrientos, 39, is charged with two counts of "offering a false instrument for filing in the first degree" In total, she has been married 10 times, with nine of her marriages occurring between 1999 and 2002. She is believed to still be married to four men.'}]


In [14]:
ARTICLE = '''Donald Trump ha dichiarato in un’intervista a NBC News che la pace tra Russia e Ucraina potrebbe non essere possibile a causa del profondo odio tra i leader e i generali, pur mantenendo una certa speranza. Ha inoltre detto di non sapere se deve rispettare la Costituzione USA e ha citato Marco Rubio e JD Vance come possibili successori alla guida del partito repubblicano. Trump ha escluso di rimuovere Jerome Powell dalla presidenza della Fed, pur criticandolo, e ha ribadito l’intenzione di mantenere o estendere alcuni dazi per incentivare la produzione negli Stati Uniti. Riguardo alla Cina, ha affermato che ora vuole un accordo commerciale equo. Infine, ha minimizzato il rischio di recessione, parlando di una fase di transizione.'''

print(summarizer(ARTICLE, max_length=100, min_length=30, do_sample=True))

[{'summary_text': 'Donald Trump ha dichiarato in un’intervista a NBC News that la pace tra Russia e Ucraina potrebbe non essere possibile a causa del profondo odio. Ha inoltre detto di non sapere se deve rispettare la Costituzione USA.'}]
